In [2]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [2]:
comments1 = pd.read_excel('../data/2_5242423354782122371.xlsx')

In [3]:
comments2 = pd.read_excel('../data/2_5242423354782122372.xlsx')

In [4]:
comments2.head()

,Platform,Country,Date,App ID,App Name,Publisher ID,Publisher Name,User,Version,Rating,Title,Review
0,iOS,United States,02/27/2018,640111933,Pixel Gun 3D,561634640,Cubic Games,lukieboy54,13.5.3,5,So awesome!!!,This is the most fun app i ever played but has...
1,iOS,United States,02/27/2018,640111933,Pixel Gun 3D,561634640,Cubic Games,BeastyTwin7,13.5.3,5,Amazing Game,Why is it so hard to get a mythical pet from b...
2,iOS,United States,02/27/2018,640111933,Pixel Gun 3D,561634640,Cubic Games,Ymartinez1748,13.5.3,5,Good game,Ti is a fun game to play the have cool weapons...
3,iOS,United States,02/27/2018,640111933,Pixel Gun 3D,561634640,Cubic Games,savaga tbh,13.5.3,5,Best app,Is a great app and everyone should get it.
4,iOS,United States,02/27/2018,640111933,Pixel Gun 3D,561634640,Cubic Games,recethan,13.5.3,5,Suggestion,I love this game and I play it all the time an...


In [20]:
comments = pd.concat([comments1[['Rating', 'Review']], comments2[['Rating', 'Review']]], axis = 0)

In [21]:
def detect_lang(x):
    try:
        return detect(x)
    except:
        return 'unknown'
comments['lang'] = comments.Review.apply(detect_lang)

In [22]:
comments.lang.value_counts()

en         19782
es         10808
ru          7937
ko          7450
pt          5119
de          4128
pl          2087
fr          1996
ja          1902
so          1834
it          1479
id          1145
tr          1126
af           990
th           974
sl           930
ca           922
vi           920
nl           803
fi           770
ro           765
tl           760
bg           748
no           692
unknown      656
hr           580
uk           490
sw           415
et           401
hu           387
mk           340
zh-cn        307
cy           288
da           284
lt           275
sk           262
sv           238
cs           184
el           159
sq           129
lv           107
zh-tw        106
ar            17
ur             3
he             2
mr             1
fa             1
Name: lang, dtype: int64

In [23]:
comments.head()

,Rating,Review,lang
0,4,Очень хорошая игра:),ru
1,5,Ich habe es vor 3 wochen runtergeladen war all...,de
2,5,Me encanta y soy manucraft,es
3,4,E destul de bun jocul dar la grafica mai trebu...,ro
4,5,Bueno me gustan los juegos de pistolas,es


In [3]:
#comments.to_csv('../data/comments_lang.csv')
comments = pd.read_csv('../data/comments_lang.csv')

In [4]:
comments_en = comments[comments.lang == 'en']
n_train = 13800
comments_en_train = comments_en.iloc[:n_train,:].sample(frac = 1)
comments_en_test = comments_en.iloc[n_train:,:].sample(frac = 1)

In [66]:
comments_en_test.to_excel('test_en.xlsx')

In [5]:
comments_en_train.shape

(13800, 4)

In [264]:
#comments_en_train.iloc[:1000,:].to_excel('manual_labels.xlsx')

In [269]:
#comments_en_train.iloc[1000:,:].to_excel('manual_unlabeled.xlsx')

In [9]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [29]:
n_samples = len(comments_en_train)
n_features = 1000
n_components = 10
n_top_words = 50

def store_top_words(model, feature_names, n_top_words):
    msg = []
    for topic_idx, topic in enumerate(model.components_):
        #message = "Topic #%d: " % topic_idx
        message = "' '".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        msg.append(message)
    res = pd.Series(msg)
    res.to_excel('lda_topics.xls')

    
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += "' '".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print("Extracting tf features for LDA...")

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                ngram_range = (1,3),
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(comments_en_train.Review)
print("done in %0.3fs." % (time() - t0))
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)
store_top_words(lda, tf_feature_names, n_top_words)

Extracting tf features for LDA...
done in 0.871s.
Fitting LDA models with tf features, n_samples=13800 and n_features=1000...
done in 11.116s.

Topics in LDA model:
Topic #0: game' 'best' 'best game' 'update' 'played' 'amazing' 'new update' 'game played' 'game best' 'new' 'world' 'app' 'sucks' 'game amazing' 'best game played' 'shooting' 'says' 'fps' 'life' 'game world' 'shooting game' 'amazing game' 'best game world' 'mobile' 'updated' 'update game' 'best shooting' 'game update' 'game best game' 'think best' 'best shooting game' 'store' 'worst' 'google play' 'fps game' 'think best game' 'liked' 'yes' 'best fps' 'recommend' 'best pixel' 'love game best' 'update sucks' 'far' 'android' 'best game play' 'best shooter' 'played life' 'seen' 'wait
Topic #1: make' 'awesome' 'guns' 'new' 'game' 'awsome' 'loved' 'update' 'weapons' 'pls' 'awesome game' 'shot' 'campaign' 've' 'armor' 'addicting' 'skins' 'bring' 'skin' 'updates' 'things' 'hope' 'rifle' 've played' 'map' 'pets' 'lots' 'make game' '

In [11]:
comments.shape

(81699, 4)

In [12]:
test_review = comments_en.Review.iloc[42]

In [13]:
test_review

'The new update is the worst pls owner of the game make it normal'

In [14]:
tf_feats = tf_vectorizer.transform([test_review])
np.argmax(lda.transform(tf_feats)[0])

2

In [15]:
lda_topics_labels = pd.read_excel('lda_topics_labels.xls')

In [16]:
lda_labels = lda_topics_labels.cat.tolist()

In [8]:
def get_lda_topic(text):
    tf_feats = tf_vectorizer.transform([text])
    return lda_labels[np.argmax(lda.transform(tf_feats)[0])]

In [9]:
get_lda_topic('Overall this is a great game but before i give it 5 stars you really need to fix a bug the matchmaking system. It keeps matching me up against a team who is almost 10 levels higher than my own, which means they get more health and have better weapons. This is making it really unfair.')

NameError: name 'tf_vectorizer' is not defined

In [6]:
val_en = pd.read_excel('validation_en.xlsx')

In [10]:
val_en.Positive.apply(get_lda_topic)

NameError: name 'tf_vectorizer' is not defined

In [ ]:
import pickle
with open('lda.pickle', 'wb') as f:
    pickle.dump(lda, f)
with open('vect.pickle', 'wb') as f:
    vect.dump(lda, f)

In [ ]:
with open('lda.pickle', 'rb') as f:
    lda = pickle.load(f)


In [37]:
comments_en_train['label'] = comments_en_train.Review.apply(get_lda_topic)

In [38]:
comments_en_train.label.value_counts()

4    4806
2    4774
3    1584
5    1327
1    1309
Name: label, dtype: int64

In [30]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = TfidfVectorizer(ngram_range = (1,3), analyzer = 'word', stop_words = 'english', min_df = 50, max_df = 0.8).fit(comments_en.Review)
feats = vect.transform(comments_en_train.Review)
labels = comments_en_train.label

AttributeError: 'DataFrame' object has no attribute 'label'

In [87]:
feats.shape

(13800, 517)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [52]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    class_names = [#'unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [90]:
m = OneVsRestClassifier(DummyClassifier()).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

           Crash       0.09      0.03      0.05      1038
Balance problems       0.34      0.18      0.24      3824
 Synchronization       0.13      0.08      0.10      1264
        Positive       0.35      0.32      0.33      3857
             Bug       0.09      0.39      0.15      1057

     avg / total       0.27      0.22      0.23     11040

test scores

                  precision    recall  f1-score   support

           Crash       0.15      0.05      0.08       271
Balance problems       0.36      0.18      0.24       950
 Synchronization       0.07      0.05      0.06       320
        Positive       0.34      0.30      0.31       949
             Bug       0.09      0.39      0.15       270

     avg / total       0.27      0.21      0.22      2760



In [91]:
m = LogisticRegression(C = 1.5).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

           Crash       0.96      0.85      0.90      1038
Balance problems       0.89      0.89      0.89      3824
 Synchronization       0.79      0.87      0.83      1264
        Positive       0.91      0.91      0.91      3857
             Bug       0.93      0.88      0.90      1057

     avg / total       0.89      0.89      0.89     11040

test scores

                  precision    recall  f1-score   support

           Crash       0.94      0.83      0.88       271
Balance problems       0.85      0.87      0.86       950
 Synchronization       0.78      0.82      0.80       320
        Positive       0.89      0.90      0.90       949
             Bug       0.91      0.86      0.88       270

     avg / total       0.87      0.87      0.87      2760



In [83]:
m = RidgeClassifier(alpha = 10).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

           Crash       0.95      0.84      0.90      1063
Balance problems       0.77      0.91      0.83      3813
 Synchronization       0.88      0.52      0.65      1249
        Positive       0.88      0.89      0.88      3855
             Bug       0.88      0.81      0.84      1060

     avg / total       0.85      0.84      0.84     11040

test scores

                  precision    recall  f1-score   support

           Crash       0.94      0.83      0.88       246
Balance problems       0.73      0.87      0.79       961
 Synchronization       0.81      0.47      0.59       335
        Positive       0.86      0.86      0.86       951
             Bug       0.86      0.81      0.83       267

     avg / total       0.82      0.81      0.80      2760



In [58]:
m = MultinomialNB(alpha = 0.01).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

           Crash       0.94      0.87      0.90      1063
Balance problems       0.78      0.87      0.82      3813
 Synchronization       0.79      0.54      0.64      1249
        Positive       0.79      0.83      0.81      3855
             Bug       0.88      0.75      0.81      1060

     avg / total       0.81      0.81      0.80     11040

test scores

                  precision    recall  f1-score   support

           Crash       0.93      0.83      0.87       246
Balance problems       0.74      0.85      0.79       961
 Synchronization       0.77      0.45      0.57       335
        Positive       0.76      0.81      0.79       951
             Bug       0.89      0.75      0.82       267

     avg / total       0.78      0.78      0.77      2760



In [93]:
def eval_classifier(input_text,model = m):
    feats = vect.transform([input_text])
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats)
    #print(class_names[prediction[0]])
    return class_names[prediction[0]]

In [96]:
val_en.Bug.apply(eval_classifier)

0    Balance problems
1    Balance problems
2    Balance problems
3    Balance problems
4     Synchronization
5    Balance problems
6     Synchronization
7     Synchronization
8            Positive
9    Balance problems
Name: Bug, dtype: object

In [22]:
from sklearn.utils import check_random_state
from sklearn.decomposition._online_lda import _dirichlet_expectation_2d
from sklearn.decomposition import NMF, LatentDirichletAllocation
class PTWGuidedLatentDirichletAllocation(LatentDirichletAllocation):

    def __init__(self, ptws=None, *args, **kwargs):
        super(PTWGuidedLatentDirichletAllocation, self).__init__(*args, **kwargs)
        self.ptws = ptws

    def _init_latent_vars(self, n_features):
        """Initialize latent variables."""

        self.random_state_ = check_random_state(self.random_state)
        self.n_batch_iter_ = 1
        self.n_iter_ = 0

        if self.doc_topic_prior is None:
            self.doc_topic_prior_ = 1. / self.n_topics
        else:
            self.doc_topic_prior_ = self.doc_topic_prior

        if self.topic_word_prior is None:
            self.topic_word_prior_ = 1. / self.n_topics
        else:
            self.topic_word_prior_ = self.topic_word_prior

        init_gamma = 100.
        init_var = 1. / init_gamma
        # In the literature, this is called `lambda`
        self.components_ = self.random_state_.gamma(
            init_gamma, init_var, (self.n_topics, n_features))

        # Transform topic values in matrix for prior topic words
        if self.ptws is not None:
            for ptw in self.ptws:
                word_index = ptw[0]
                word_topic_values = ptw[1]
                self.components_[:, word_index] *= word_topic_values

        # In the literature, this is `exp(E[log(beta)])`
        self.exp_dirichlet_component_ = np.exp(
            _dirichlet_expectation_2d(self.components_))

In [23]:
from sklearn.feature_extraction import stop_words
sw = list(stop_words.ENGLISH_STOP_WORDS)
sw.append('game')
sw.append('games')
sw.append('good')
sw.append('great')

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from __future__ import print_function
from time import time
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                ngram_range = (1,3),
                                max_features=1000,
                                stop_words=sw)
t0 = time()
tf = tf_vectorizer.fit_transform(comments_en_train.Review)

In [36]:
tf_vectorizer.vocabulary_['cool']

163

In [26]:
(np.finfo(np.float64).max/4)

4.4942328371557893e+307

In [27]:
# List with prior topic words as tuples
# (word index, [topic values])
def prior_dist(n_topic):
    dist = [0.,0.,0.,0.,0.]
    dist[n_topic] = (np.finfo(np.float64).max/4)
    return dist
prior_topic_words = [
    (tf_vectorizer.vocabulary_['crash'],prior_dist(0)),
    (tf_vectorizer.vocabulary_['crashes'],prior_dist(0)),
    (tf_vectorizer.vocabulary_['crashing'],prior_dist(0)),
    (tf_vectorizer.vocabulary_['balance'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['balanced'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['unbalanced'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['overpowered'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['op'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['match'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['unfair'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['prices'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['weapons cost'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['powerful'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['fair'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['hard'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['higher'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['level'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['levels'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['buy gems'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['matchmaking'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['beast'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['level 27'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['pay win'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['powered'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['guns cost'],prior_dist(1)),
    (tf_vectorizer.vocabulary_['lost'],prior_dist(2)),
    (tf_vectorizer.vocabulary_['lose'],prior_dist(2)),
    (tf_vectorizer.vocabulary_['trophies'],prior_dist(2)),
    (tf_vectorizer.vocabulary_['losing'],prior_dist(2)),
    (tf_vectorizer.vocabulary_['deleted'],prior_dist(2)),
    (tf_vectorizer.vocabulary_['awesome'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['amazing'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['good'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['awesome cool'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['awesome graphics'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['good gameplay'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['good time'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['good job'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['fun love'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['fun great'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['really like'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['good best'],prior_dist(3)),
    #(tf_vectorizer.vocabulary_['good graphics'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['best'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['cool'],prior_dist(3)),
    (tf_vectorizer.vocabulary_['bug'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['bugs'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['lag'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['lags'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['lagging'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['update'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['google play'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['fix'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['fix'],prior_dist(0)),
    (tf_vectorizer.vocabulary_['fix bug'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['fix bug'],prior_dist(0)),
    (tf_vectorizer.vocabulary_['google'],prior_dist(4)),
    (tf_vectorizer.vocabulary_['got banned'],prior_dist(4))
]

# Example (word at index 3000 belongs to topic with index 0)
#prior_topic_words.append(
#    (3000, [(np.finfo(np.float64).max/4),0.,0.,0.,0.])
#)

In [30]:
lda = PTWGuidedLatentDirichletAllocation(n_topics=5, max_iter=50,
                                learning_method='online',
                                learning_offset=15.,
                                random_state=0,
                                ptws = prior_topic_words)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)
store_top_words(lda, tf_feature_names, n_top_words)

/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


done in 55.543s.

Topics in LDA model:
Topic #0: death' 'definitely' 'delete' 'game make' 'game love' 'game lots' 'game lot' 'game like' 'game life' 'game just' 'game ive' 'game im' 'game guys' 'game guns' 'game great' 'zombie' 'game addicting' 'game got' 'game good' 'game game' 'game fun play' 'game fun' 'game download' 'game dont' 'game don' 'game cool' 'game better' 'game best game' 'game best' 'game awsome' 'game mobile' 'game mode' 'game modes' 'game needs' 'game years' 'game world' 'game wish' 'game want' 'game ve played' 'game ve' 'game used' 'game update' 'game time' 'game thank' 'game stars' 'game really good' 'game really fun' 'game really' 'game problem' 'game plz
Topic #1: player' 'installed' 'trash' 'mucho' 'love graphics' 'graphics good' 'fun' 'internet' 'best game' 'chest' 'new' 'best fps' 'instead' 'play time' 'new weapons' 'play store' 'totally' 'love game fun' 'best mobile' 'weak' 'gone' 'games' 'game life' 'game like' 'game ive' 'game just' 'game im' 'zombie' 'game g

/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:782: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)


In [31]:
def get_lda_topic(text):
    tf_feats = tf_vectorizer.transform([text])
    return np.argmax(lda.transform(tf_feats)[0])

In [32]:
get_lda_topic('Overall this is a great game but before i give it 5 stars you really need to fix a bug the matchmaking system. It keeps matching me up against a team who is almost 10 levels higher than my own, which means they get more health and have better weapons. This is making it really unfair.')

0

In [33]:
val_en.Bug.apply(get_lda_topic)

0    2
1    1
2    2
3    2
4    1
5    0
6    0
7    1
8    0
9    0
Name: Bug, dtype: int64

In [238]:
y_true = []
y_pred = []
for i in range(5):
    y_true.append([i]*10)
    y_pred.append(val_en.iloc[:,i].apply(get_lda_topic))
y_true = np.array(y_true).flatten()
y_pred = np.array(y_pred).flatten()

In [239]:
class_names = [#'unknown',
    'Crash',
    'Balance problems',
    'Synchronization',
    'Positive',
    'Bug']
print(classification_report(y_true, y_pred, target_names = class_names))

                  precision    recall  f1-score   support

           Crash       0.35      0.60      0.44        10
Balance problems       0.50      0.40      0.44        10
 Synchronization       1.00      0.50      0.67        10
        Positive       0.50      0.50      0.50        10
             Bug       0.60      0.60      0.60        10

     avg / total       0.59      0.52      0.53        50



In [240]:
comments_en_train['label'] = comments_en_train.Review.apply(get_lda_topic)

In [241]:
comments_en_train.label.value_counts()

0    7644
3    4396
4     779
1     725
2     256
Name: label, dtype: int64

In [242]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,3), analyzer = 'word', stop_words = sw, min_df = 2, max_df = 0.95).fit(comments_en.Review)
feats = vect.transform(comments_en_train.Review)
labels = comments_en_train.label

In [243]:
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [256]:
m = LogisticRegression(C = 1).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

           Crash       1.00      1.00      1.00      6085
Balance problems       1.00      0.97      0.98       580
 Synchronization       1.00      0.99      0.99       199
        Positive       1.00      1.00      1.00      3562
             Bug       1.00      0.99      0.99       614

     avg / total       1.00      1.00      1.00     11040

test scores

                  precision    recall  f1-score   support

           Crash       0.98      1.00      0.99      1559
Balance problems       0.92      0.81      0.86       145
 Synchronization       0.94      0.60      0.73        57
        Positive       0.99      1.00      0.99       834
             Bug       0.94      0.93      0.93       165

     avg / total       0.98      0.98      0.97      2760



In [265]:
from sklearn.linear_model import LassoCV
m = OneVsRestClassifier(LassoCV()).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

           Crash       0.95      1.00      0.97      6085
Balance problems       0.99      0.65      0.78       580
 Synchronization       0.98      0.69      0.81       199
        Positive       0.99      1.00      1.00      3562
             Bug       0.98      0.84      0.90       614

     avg / total       0.97      0.97      0.96     11040

test scores

                  precision    recall  f1-score   support

           Crash       0.94      1.00      0.97      1559
Balance problems       0.93      0.52      0.67       145
 Synchronization       0.94      0.60      0.73        57
        Positive       0.98      1.00      0.99       834
             Bug       0.96      0.79      0.87       165

     avg / total       0.95      0.95      0.95      2760



In [266]:
def eval_classifier(input_text,model = m):
    feats = vect.transform([input_text])
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats)
    #print(class_names[prediction[0]])
    return prediction[0]

In [267]:
y_true = []
y_pred = []
for i in range(5):
    y_true.append([i]*10)
    y_pred.append(val_en.iloc[:,i].apply(eval_classifier))
y_true = np.array(y_true).flatten()
y_pred = np.array(y_pred).flatten()

In [268]:
class_names = [#'unknown',
    'Crash',
    'Balance problems',
    'Synchronization',
    'Positive',
    'Bug']
print(classification_report(y_true, y_pred, target_names = class_names))

                  precision    recall  f1-score   support

           Crash       0.25      0.60      0.35        10
Balance problems       0.60      0.30      0.40        10
 Synchronization       1.00      0.40      0.57        10
        Positive       0.50      0.50      0.50        10
             Bug       0.43      0.30      0.35        10

     avg / total       0.56      0.42      0.44        50

